In [1]:
import numpy as np
runif=np.random.uniform
from scipy.special import gamma
import time
from collections import deque

# Generate arrivals and process times

i copied the professor matlab code into python.


This generates the arrivals time and the service times

In [2]:
###here i just write in python the matlab script from the prof

ntry=1000
n=100000
mTv=[]
ratioTv=[]
mXv=[]
ratioXv=[]
telapsedv=[]
rhosimv=[]
for i in range(ntry):
    #start timer
    start=time.time()
    
    #arrival time
    T0=1
    #average Y
    Ymed=10
    #prob q
    q=3/5
    #average time arrival
    ET=T0+(1-q)*Ymed
    #variance of time arrival
    ET2=T0**2+(1-q)*(2*Ymed**2+2*T0*Ymed)
    #
    CT=np.sqrt(ET2-ET**2)/ET
    #random vector
    v=np.round(q-runif(0,1,n))
    #vector of generated arrival times
    Tv=T0-(1-v)*np.log(runif(0,1,n))*Ymed
    #its mean
    mT=np.mean(Tv)
    #its std
    sT=np.std(Tv)
    # std over mean
    cT = sT/mT
    #confidence interval of T
    ICT = 1.96*sT/np.sqrt(n)
    # compute ratio
    ratioT=ICT/mT
    # save to the vector
    mTv.append(mT)
    # save to the vector
    ratioTv.append(ratioT)

    #processing time expected value
    EX=99
    #weibull distro parameters
    alfa=1/2
    beta=EX/gamma(1+1/alfa)
    EX2=beta**2*gamma(1+2/alfa)
    CX=np.sqrt(EX2-EX**2)/EX
    #generate vector of processing time 
    Xv=np.maximum(1,np.minimum(100*EX, np.round(beta*(-np.log(runif(0,1,n)))**(1/alfa))))
    #compute metrics on the processing times
    mX=np.mean(Xv)
    sX=np.std(Xv)
    cX=sX/mX
    ICX=1.96*sX/np.sqrt(n)
    ratioX=ICX/mX
    mXv.append(mX)
    ratioXv.append(ratioX)
    
    #utilization coeff
    
    #N server
    Nse=20
    rho=EX/(Nse*ET)
    rhosim=mX/Nse/mT
    rhosimv.append(rhosim)
    
    end = time.time()
    telapsed= end - start
    telapsedv.append(telapsed)
    
# print(f'estimated average inter arrival time are \n{mTv}\n')
# print(f'ratio of 95% CI to estimated average of interarrival times are \n{ratioTv}\n')
# print(f'estimated average service time are \n{mXv}\n')
# print(f'ratio of 95% CI of estimated average of service time \n{ratioXv}\n')
# print(f'estimated utilization coefficient \n{rhosimv}\n')
# print('')
# print(f'average computation time {1000*np.mean(telapsedv)} ms')

# POD

with the generated times i run a POD

In [23]:
#N servers
N=20
#start with empty server
server= [deque() for i in range(N)]
#memory to know how many task in queue for each server
memory= [0]*N
#arrival times of tasks (as cumulative sum of arrival times)
task_arrival=np.cumsum(mTv)
#service times
service_times=mXv
#POD parameter
d=3

for time, serviceT in zip(task_arrival, service_times):
    
    #check if at time=time there are task that are done,
    #if they are done, delete them from the queue
    for i in range(len(server)):
        try:
            if server[i][0]<time:
                server[i].popleft()
        except:
            continue
    
    #POD, select 3 servers and check how many task in queue
    for i in range(d):
        #choise random server
        rnd_server=np.random.choice(list(range(N)))  
        #save to memory the len of the queue of the chosen server
        memory[rnd_server]=len(server[rnd_server]) 
        
    #save minimun length queue value
    min_value = min(memory)
    #save the index(es) of the server(s) with minimum length queue
    to_choose=[i for i, x in enumerate(memory) if x == min_value]
    #select randomly over servers with queue with min len over the d sever extracted
    selected_server=np.random.choice(to_choose)
    
    #update server, with new task saved as with time T the task will be over (current time+service time)
    server[selected_server].append(serviceT+time)
    #update memory
    memory[selected_server]+=1

print(f'finished at time {time}')

finished at time 10001.529666403494


This is the state of the servers at the end of the simulation.

each servers has a queue of task, each task of the queue has the time at which it will be over 

In [4]:
server

[deque([10100.245286403495]),
 deque([10059.99415481081]),
 deque([10081.208126684454]),
 deque([]),
 deque([10011.801792820945]),
 deque([]),
 deque([10071.308329160802]),
 deque([]),
 deque([]),
 deque([]),
 deque([]),
 deque([]),
 deque([10039.532110919074]),
 deque([10021.27230687859]),
 deque([10090.56486772017]),
 deque([]),
 deque([10049.853352494276]),
 deque([]),
 deque([10029.926021346122]),
 deque([])]

The memory keeps track of the length of the queues

In [5]:
memory

[1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0]

# JBT-d

In [27]:
#N servers
N=20
#start with empty server
server= [deque() for i in range(N)]
#memory to know how many task in queue for each server
memory= [0]*N
#arrival times of tasks (as cumulative sum of arrival times)
task_arrival=np.cumsum(mTv)
#service times
service_times=mXv
#POD parameter
d=3
T=1
thousand=1

for time, serviceT in zip(task_arrival, service_times):

    for i in range(len(server)):
        try:
            if server[i][0]<time:
                server[i].popleft()
                memory[i]-=1
        except:
            continue
            
    #check if 1000 time units passed
    if time/1000>thousand:
        #update threshold
        new_T=[]
        for i in range(d):
            #choise random server
            rnd_server=np.random.choice(list(range(N)))  
            #save to possible new threshold the len of queue of selected server
            new_T.append(len(server[rnd_server]))
        #save minimun length queue value as new
        min_value = min(new_T)
        T=min_value
        thousand+=1
        
    #select a random server over the ones with queue<T
    to_choose=[i for i in range(len(memory)) if memory[i]<=T]
    
    if to_choose:
        selected_server=np.random.choice(to_choose)
    else:
        selected_server=np.random.choice(list(range(N)))
             
    #update server, with new task saved as with time T the task will be over (current time+service time)
    server[selected_server].append(serviceT+time)
    memory[selected_server]+=1

print(f'finished at time {time}')

finished at time 10001.529666403494


In [28]:
server

[deque([]),
 deque([10049.853352494276]),
 deque([10039.532110919074]),
 deque([]),
 deque([]),
 deque([]),
 deque([10090.56486772017]),
 deque([]),
 deque([10059.99415481081]),
 deque([10100.245286403495]),
 deque([10011.801792820945]),
 deque([10071.308329160802]),
 deque([10021.27230687859]),
 deque([]),
 deque([10081.208126684454]),
 deque([]),
 deque([]),
 deque([]),
 deque([10029.926021346122]),
 deque([])]

In [29]:
memory

[0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0]

# JSQ

Just join the shortest line

In [3]:
#N servers
N=20
#start with empty server
server= [deque() for i in range(N)]
#memory to know how many task in queue for each server
memory= [0]*N
#arrival times of tasks (as cumulative sum of arrival times)
task_arrival=np.cumsum(mTv)
#service times
service_times=mXv

for time, serviceT in zip(task_arrival, service_times):
    
    #check if at time=time there are task that are done,
    #if they are done, delete them from the queue
    for i in range(len(server)):
        try:
            if server[i][0]<time:
                server[i].popleft()
        except:
            continue
    
    for i in range(N): 
        #save to memory the len of the queue of the chosen server
        memory[i]=len(server[i]) 
        
    #save minimun length queue value
    min_value = min(memory)
    #save the index(es) of the server(s) with minimum length queue
    to_choose=[i for i, x in enumerate(memory) if x == min_value]
    #select randomly over servers with queue with min len over the d sever extracted
    selected_server=np.random.choice(to_choose)
    
    #update server, with new task saved as with time T the task will be over (current time+service time)
    server[selected_server].append(serviceT+time)
    #update memory
    memory[selected_server]+=1

print(f'finished at time {time}')

finished at time 10000.451383075728


In [4]:
memory

[0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1]

In [5]:
server

[deque([]),
 deque([]),
 deque([10089.626086928645]),
 deque([10009.290412280567]),
 deque([]),
 deque([10070.640649911515]),
 deque([10059.371051643204]),
 deque([10080.19415713407]),
 deque([10019.868615194475]),
 deque([10031.198548039894]),
 deque([]),
 deque([10098.111123075727]),
 deque([]),
 deque([]),
 deque([]),
 deque([]),
 deque([10039.595181383609]),
 deque([]),
 deque([]),
 deque([10049.241794940277])]

### just notes and garbage, you can ignore this

In [9]:
"""#this is the function of the pod
def POD(d,N,server):
    #i assume that we start not knowing the queue in all server
    memory=[0]*N
    #here i choose d random servers and save their queue to the memory
    for i in range(d):
        #choice a server randomly over a list of N
        rnd_server=random.choice(list(range(N)))
        #save it's queue length to memory
        memory[rnd_server]=server[rnd_server]
    #save minimun length value
    min_value = min(memory)
    #save the index of the server with minimum length queue
    to_choose=[i for i, x in enumerate(memory) if x == min_value]
    #select randomly over server with queue with min len over the d sever extracted
    selected_server=random.choice(to_choose)
    #update memory, now that the chosen server has 1 more user in queue
    memory[selected_server]+=1
    return selected_server"""

"#this is the function of the pod\ndef POD(d,N,server):\n    #i assume that we start not knowing the queue in all server\n    memory=[0]*N\n    #here i choose d random servers and save their queue to the memory\n    for i in range(d):\n        #choice a server randomly over a list of N\n        rnd_server=random.choice(list(range(N)))\n        #save it's queue length to memory\n        memory[rnd_server]=server[rnd_server]\n    #save minimun length value\n    min_value = min(memory)\n    #save the index of the server with minimum length queue\n    to_choose=[i for i, x in enumerate(memory) if x == min_value]\n    #select randomly over server with queue with min len over the d sever extracted\n    selected_server=random.choice(to_choose)\n    #update memory, now that the chosen server has 1 more user in queue\n    memory[selected_server]+=1\n    return selected_server"